<a href="https://colab.research.google.com/github/digicornio/ciclopeia/blob/main/ciclopeia_wandb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Install requirements

In [ ]:
# clone and install yolov5
%%capture
!git clone https://github.com/ultralytics/yolov5.git
%cd /content/yolov5
!pip install -r requirements.txt

# install wandb for training metrics
!pip install wandb

In [ ]:
# import core and utilities libraries
import os
import random
import shutil
import numpy as np
from IPython.display import Image, clear_output 

# import ai and training metric libraries
import torch
from yolov5 import utils

# validate CUDA is enabled
display = utils.notebook_init()

YOLOv5 🚀 v6.0-244-g9cf80b7 torch 1.10.0+cu111 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 43.2/107.7 GB disk)


# Step 2: Dataset preparation

In [ ]:
# define dataset source and target after split
dataset_source = '/content/drive/MyDrive/Saturdays_AI/RED/DEMODAY_PROJECT/nuevo-dataset/' # change to your path 
dataset_target = '/content/yolov5/custom_dataset/'

In [ ]:
# list and compar images against labels
labels = [os.path.join(dataset_source, x) for x in os.listdir(dataset_source) if x[-3:] == "txt"]
images = [os.path.join(dataset_source, x) for x in os.listdir(dataset_source) if x[-3:] == "jpg"]
print(f'| Total: {len(labels)} labels & {len(images)} images from: {dataset_source}')

| Total: 1045 labels & 1045 images from: /content/drive/MyDrive/Saturdays_AI/RED/DEMODAY_PROJECT/nuevo-dataset/


In [ ]:
# function to copy images and labels from source (gdrive) to target (colab)
def copy_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.copy(f, destination_folder)
        except:
            print(f)
            assert False   

In [ ]:
from sklearn.model_selection import train_test_split
# training, validation & test splits
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size = 0.2, random_state = 1)
val_images, test_images, val_labels, test_labels = train_test_split(val_images, val_labels, test_size = 0.5, random_state = 1)

In [ ]:
# creates dataset_target folders to copy images and labeles from dataset source
# !mkdir /content/yolov5/custom_dataset custom_dataset/train custom_dataset/val custom_dataset/test
!mkdir {dataset_target} {dataset_target + 'train'} {dataset_target + 'val'} {dataset_target + 'test'} 

In [ ]:
# copy images and labels into dataset_target folders
copy_files_to_folder(train_images + train_labels, dataset_target + 'train')
copy_files_to_folder(val_images + val_labels, dataset_target + 'val')
copy_files_to_folder(test_images + test_labels, dataset_target + 'test')

# Step 3: Train YOLOv5 model

In [ ]:
import wandb
# login into wandb
wandb.login()
# configure wandb project and user
wandb.init(project="ciclopeiav20", entity="digicornio")


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: digicornio (use `wandb login --relogin` to force relogin)


In [ ]:
import json
import yaml
# creates .yaml to define clases and dataset locations
data={

    'names':['$1USD', '$2USD', '$5USD', '$10USD', '$20USD', '$50USD', '$100USD'],
    'nc': 7,
    'train': "custom_dataset/train",
    'val': "custom_dataset/val",
 
    # 'depth_multiple': 0.33,
    # 'width_multiple': 0.50,

    # 'anchors':
    # - [10,13, 16,30, 33,23] 
    # - [30,61, 62,45, 59,119]
    # - [116,90, 156,198, 373,326] 

    # 'backbone':
    # [[-1, 1, Focus, [64, 3]],
    # [-1, 1, Conv, [128, 3, 2]],
    # [-1, 3, Bottleneck, [128]],
    # [-1, 1, Conv, [256, 3, 2]],
    # [-1, 9, BottleneckCSP, [256]],
    # [-1, 1, Conv, [512, 3, 2]], 
    # [-1, 9, BottleneckCSP, [512]],
    # [-1, 1, Conv, [1024, 3, 2]],
    # [-1, 1, SPP, [1024, [5, 9, 13]]],
    # [-1, 6, BottleneckCSP, [1024]],
    # ]

    # 'head':
    # [[-1, 3, BottleneckCSP, [1024, False]],
    # [-1, 1, nn.Conv2d, [na * (nc + 5), 1, 1, 0]],
    # [-2, 1, nn.Upsample, [None, 2, "nearest"]],
    # [[-1, 6], 1, Concat, [1]],
    # [-1, 1, Conv, [512, 1, 1]],
    # [-1, 3, BottleneckCSP, [512, False]],
    # [-1, 1, nn.Conv2d, [na * (nc + 5), 1, 1, 0]],
    # [-2, 1, nn.Upsample, [None, 2, "nearest"]],
    # [[-1, 4], 1, Concat, [1]],
    # [-1, 1, Conv, [256, 1, 1]],
    # [-1, 3, BottleneckCSP, [256, False]],
    # [-1, 1, nn.Conv2d, [na * (nc + 5), 1, 1, 0]],

    # [[], 1, Detect, [nc, anchors]],
    # ]

 }
with open('data/usd_bills.yaml', 'w') as outfile:
    yaml.dump(data, outfile)

In [ ]:
# !python utils/loggers/wandb/log_dataset.py --project ciclopeia --data data/usd_bills.yaml

In [ ]:
!python train.py \
  --img 640 \
  --batch 16 \
  --epochs 10 \
  --data data/usd_bills.yaml \
  --project ciclopeia \
  --upload_dataset \
  --bbox_interval 1 \
  --save-period 1

wandb: Currently logged in as: digicornio (use `wandb login --relogin` to force relogin)
train: weights=yolov5s.pt, cfg=, data=data/usd_bills.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=10, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=ciclopeia, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=1, local_rank=-1, entity=None, upload_dataset=True, bbox_interval=1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-244-g9cf80b7 torch 1.10.0+cu111 CPU

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h

Exception: ignored

# Step 4: Evaluate YOLOv5 model performance


In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 416 --conf 0.1 --source {dataset.location}/test/images

In [ ]:
#display inference on ALL test images

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

In [ ]:
#export your model's weights for future use
from google.colab import files
files.download('./runs/train/exp/weights/best.pt')